In [7]:
import spacy
nlp = spacy.load('en_core_web_md')
import wikipedia
import clarifai
import telebot
import config
import logging
import emoji
import random
import pickle
from telegram import InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Updater, CommandHandler, CallbackQueryHandler
#import pb
#import datetime
#import pytz
#import json
import traceback
import spacy
import wikipedia
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
from clarifai.rest import ClarifaiApp, Image
import pickle
with open("all_unique.txt", "rb") as fp:   # Unpickling
    all_unique = pickle.load(fp)
all_unique = [item.lower() for item in all_unique]

#bot = telebot.TeleBot(config.TOKEN)
#bot.polling(none_stop=True)
def keyphrase(doc):
    for t in doc:
        if t.dep_ == 'pobj' and (t.pos_ == 'NOUN' or t.pos_ == 'PROPN'): #add adjective
            return (' '.join([child.text for child in t.lefts]) + ' ' + t.text).lstrip()
    for i, t in enumerate(doc):
        if t.dep_ == 'nsubj' and (t.pos_ == 'NOUN' or t.pos_ == 'PROPN'):
            if  ((doc[i - 1].dep_ == "amod") or (doc[i - 1].dep_ == "compound") or (doc[i - 1].dep_ == "conj")):
                return doc[i - 1].text + ' ' + t.text
            return t.text + ' ' + t.head.text
    for i, t in enumerate(doc):
        if t.dep_ == 'dobj' and (t.pos_ == 'NOUN' or t.pos_ == 'PROPN'):
            if  ((doc[i - 1].dep_ == "amod") or (doc[i - 1].dep_ == "compound") or (doc[i - 1].dep_ == "conj")):
                return doc[i - 1].text + ' ' + t.text
            return t.text
    for i, t in enumerate(doc):
        if t.dep_ == 'attr' and (t.pos_ == 'NOUN' or t.pos_ == 'PROPN'):
            if  ((doc[i - 1].dep_ == "amod") or (doc[i - 1].dep_ == "conj") or (doc[i - 1].dep_ == "cmpound")):
                return doc[i - 1].text + ' ' + t.text
            return t.head.text  + ' ' + t.text
    return doc.text



def photo_tags(filename):
    app = ClarifaiApp(api_key='API_KEY')
    image = Image(file_obj=open(filename, 'rb'))
    model = app.models.get(model_name = "moderation")
    response = model.predict([image])
    value_1 = response['outputs'][0]['data']['concepts'][0]['name'], response['outputs'][0]['data']['concepts'][0]['value']
    if ( (value_1[0] != "safe") and (value_1[1] >= 0.5) ):
        flag = 1
        return "Sorry, this image has been flagged as NSFW. That means we probably shouldn't be looking at this. \U0001F97A", flag
    flag = 0
    model = app.public_models.general_model
    image = Image(file_obj=open(filename, 'rb'))
    response = model.predict([image])
    concepts = response['outputs'][0]['data']['concepts']
#     for concept in concepts:
#         if concept['name'] == 'texture': #texture and patterns model
#             patt_model = app.models.get(model_name = 'Textures & Patterns')
#             result = patt_model.predict([image])
#             first_concept = result['outputs'][0]['data']['concepts'][0]['name']
#             return first_concept, flag
    return response['outputs'][0]['data']['concepts'][0]['name'], flag



def wiki(concept):

    #nlp = spacy.load('en_core_web_md')
    wiki_resp = wikipedia.search(concept)
    #temp = wiki_resp[0]
    top_10 = wiki_resp
    top_10 = [item.lower() for item in top_10]
    
    science_related = []
    for item in top_10:
        if item in all_unique:
            science_related.append(item)
            
    if science_related != []:
        wiki_resp = wikipedia.page(science_related[0], auto_suggest = False)

        #wiki_resp = wikipedia.page(science_related[0])
        return wikipedia.summary(science_related[0], sentences=3, auto_suggest = False), wiki_resp.images[0], \
    science_related[1:]
    else:
        flag = 0
        return "Sorry! that doesn't fall under our list of available topics. \U0001F61E", 0, 0
    wiki_resp = wikipedia.page(wiki_resp[0])
    doc = nlp(wiki_resp.content)
#     if len(concept.split()) == 1:
#         for sent in doc.sents:
#             for t in sent:
#                 if t.text == concept and t.dep_ == 'dobj':
#                     return sent.text
    #return list(doc.sents)[0:3]
    
    #return wikipedia.summary(temp, sentences=3, auto_suggest = False)

def wiki_photo(concept):
    #nlp = spacy.load('en_core_web_md')
    
    wiki_resp = wikipedia.search(concept)
    temp = wiki_resp[0]
    wiki_resp = wikipedia.page(wiki_resp[0], auto_suggest = False)
    #doc = nlp(wiki_resp.content)
#     if len(concept.split()) == 1:
#         for sent in doc.sents:
#             for t in sent:
#                 if t.text == concept and t.dep_ == 'dobj':
#                     return sent.text
    #return list(doc.sents)[0:3]
    
    return wiki_resp.images[0]

def start(update, context):
    update.message.reply_text("Hi! I'm Pat, the Science Robot \U0001F916."\
                              " I'm designed to help you with all of your Science-related \U0001F9EC questions." \
                              "\n\nAsk me anything about everything in the universe \U0001F30C! I'll do my best to help you out. \U0001F607" \
                              "\nYou can even try sending me a picture \U0001F5BC and I'll help you figure out what that is!" \
                              "\n\nType /help \U0001F91D if you aren't sure what to do." \
                             "\n\nType /commands \U0001F44A if you want to see everything that I can do.")

def help(update, context):
    update.message.reply_text("\nYou can just type in any questions \U0001F914 that you have for me." \
                             "\nIf you're tired or lazy \U0001F62A just like me, try writing only the topic that you want to search!" \
                              "\n\nBut please be patient with me. \U0001F623 I need some time \U0001F55B to come up with your answers!" \
                              "\n\nEx.) noble gas \U0001F934")
    keyboard = [
        [
            InlineKeyboardButton("Message me!", url = 't.me/DarthPhonsus'),
    ]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    update.message.reply_text("Feel free to tell me about any bugs \U0001F41B or if you want some features \U0001F4C8 added! ",\
                              reply_markup = reply_markup)

def bash(update, context):
    keyboard = [
        [
            InlineKeyboardButton("Meet the EskweTeam!", url = 'https://www.eskwelabs.com/'),
    ]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    update.message.reply_text("Thank you to Bash and the EskweFam \U0001F465 for helping Team Pat build me <3!", \
                              reply_markup = reply_markup)

def commands(update, context):
    update.message.reply_text("\U0001F91D /help - if you're not quite sure how to use me.\n\n\n" \
                             "\U0001F44A /commands - if you want to see everything that I can do.\n\n\n" \
                             "\U0001F9DE /random - if you want to learn about a random science topic!\n\n\n" \
                             "\U0000262F /constants - if you want to check out some physical constants!\n\n\n" \
                             "\U0001F31E /start - if you want to start over from the beginning")
    
def constants(update, context):
    update.message.reply_text("\U0001F300 Gravitational constant: G = 6.67408 × 10\u207b\u00b9\u00b9 m\u00b3 kg\u207b\u00b9 s\u207b\u00b2" \
                             "\n\n\U00002B07 Acceleration due to gravity: a = 9.81 ms\u207b\u00b2" \
                             "\n\n\U0001F534 Electron charge: e = -1.60217662 × 10\u207b\u00b9\u2079 C" \
                             "\n\n\U0001F535 Proton charge: p = 1.60217662 × 10\u207b\u00b9\u2079 C" \
                             "\n\n\U000026D4 Mass of electron: m\u2091 = 9.10938356 × 10\u207b\u00b3\u00b9 kg" \
                             "\n\n\U00002795 Mass of proton: m\u209a = 1.6726219 × 10\u207b\u00b2\u2077  kg" \
                             "\n\n\U0001F4A1 Speed of light (vacuum): c = 2.99792458 × 10\u2078 m/s." \
                             "\n\n\U0001F9EF Gas constant: R = 8.314 J mol\u207b\u00b9 K\u207b\u00b9" \
                             "\n\n\U0001F9EB Boltzmann’s constant: k = 1.38064852 × 10\u207b\u00b2\u00b3 m\u00b2 kg s\u207b\u00b2 K\u207b\u00b9" \
                             "\n\n\U0001F9D1 Avogadro's Number: N\u2090 = 6.02214076 × 10\u00b2\u00b3")

def random_topic(update, context):
    #with open("all_unique.txt", "rb") as fp:   # Unpickling
    #    all_unique = pickle.load(fp)
    #all_unique = [item.lower() for item in all_unique]
    msg = random.choice(all_unique)
    #nlp = spacy.load('en_core_web_md')
    doc = nlp(msg)
    concept = keyphrase(doc)
    if concept != False:
        summary, images, suggest = wiki(concept)
        update.message.reply_text(summary)
        if images != 0:
            update.message.reply_text(images)

        if suggest != 0:
            if suggest != []:
                update.message.reply_text("You might wanna check out these topics too! \U0001F575")
                for word in suggest:
                    update.message.reply_text(word)
        if images == 0:
            keyboard = [
        [
            InlineKeyboardButton("Message me!", url = 't.me/DarthPhonsus'),
    ]
    ]
            reply_markup = InlineKeyboardMarkup(keyboard)
            update.message.reply_text('Go ahead and tell me if you think this is a topic that I should know about! \U0001F9E0',\
                              reply_markup = reply_markup)
        #update.message.reply_text(wiki_photo(concept))
    else:
        update.message.reply_text('Please rephrase your question.')


    #reply_markup = InlineKeyboardMarkup(keyboard)
def text_msg(update, context):
    msg = update.message.text
    #nlp = spacy.load('en_core_web_md')
    doc = nlp(msg)
    concept = keyphrase(doc)
    if concept != False:
        summary, images, suggest = wiki(concept)
        update.message.reply_text(summary)
        if images != 0:
            update.message.reply_text(images)

        if suggest != 0:
            if suggest != []:
                update.message.reply_text("You might wanna check out these topics too! \U0001F575")
                for word in suggest:
                    update.message.reply_text(word)
        if images == 0:
            keyboard = [
        [
            InlineKeyboardButton("Message me!", url = 't.me/DarthPhonsus'),
    ]
    ]
            reply_markup = InlineKeyboardMarkup(keyboard)
            update.message.reply_text('Go ahead and tell me if you think this is a topic that I should know about! \U0001F9E0',\
                              reply_markup = reply_markup)
        #update.message.reply_text(wiki_photo(concept))
    else:
        update.message.reply_text('Please rephrase your question.')

        
def photo(update, context):
    photo_file = update.message.photo[-1].get_file()
    filename = '{}.jpg'.format(photo_file.file_id)
    photo_file.download(filename)
    
    concept, flag = photo_tags(filename)
    if flag == 1:
        update.message.reply_text(concept)
    else:
        #update.message.reply_text(wiki(concept))
        try:
            summary, images, suggest = wiki(concept)
            update.message.reply_text(summary)
            if images != 0:
                update.message.reply_text(images)

            if suggest != 0:
                if suggest != []:
                    update.message.reply_text("You might wanna check out these topics too! \U0001F575")
                    for word in suggest:
                        update.message.reply_text(word)
            #update.message.reply_text(wiki(concept))
            if images == 0:
                keyboard = [
        [
                InlineKeyboardButton("Message me!", url = 't.me/DarthPhonsus'),
    ]
    ]
                reply_markup = InlineKeyboardMarkup(keyboard)
                update.message.reply_text('Go ahead and tell me if you think this is a topic that I should know about! \U0001F9E0',\
                              reply_markup = reply_markup)
        except wikipedia.exceptions.DisambiguationError as e:
            update.message.reply_text("Sorry! That photo doesn't work for us. Try something else!" )
        
        
def main():
    updater = Updater("TOKEN", use_context=True)
    disp = updater.dispatcher
    disp.add_handler(CommandHandler("bash", bash))
    disp.add_handler(CommandHandler("start", start))
    disp.add_handler(CommandHandler("help", help))
    disp.add_handler(CommandHandler("random", random_topic))
    disp.add_handler(CommandHandler("commands", commands))
    disp.add_handler(CommandHandler("constants", constants))
    disp.add_handler(MessageHandler(Filters.text, text_msg))
    disp.add_handler(MessageHandler(Filters.photo, photo))
    updater.start_polling()
    updater.idle()
if __name__ == '__main__':
    main()

